# **Mozart**: The ML Models Gradio Demo:

Using Gradio 3.0, this demo is designed to showcase the backend of the Platform.

-Bharath Raj

##Installing Dependencies:

Dependencies for Colab:

In [ ]:
from google.colab import output

nvidia_output = !nvidia-smi --query-gpu=memory.total --format=noheader,nounits,csv
gpu_memory = int(nvidia_output[0])
if gpu_memory < 14000:
  output.eval_js('new Audio("https://upload.wikimedia.org/wikipedia/commons/0/05/Beep-09.ogg").play()')
  warning_string = f"--> GPU check: ONLY {gpu_memory} MiB available: Please use low quality or low res <--"
  print(warning_string)
  output.eval_js('alert("Warning - low GPU (see message)")')
else:
  print(f"GPU check: {gpu_memory} MiB available: this should be fine")

from IPython.utils import io
with io.capture_output() as captured:
  !pip install torch==1.9.0+cu102 torchtext==0.10.0 torchvision==0.10.0+cu102 torch-optimizer==0.1.0 -f https://download.pytorch.org/whl/torch/ -f https://download.pytorch.org/whl/torchvision/
  !git clone https://github.com/openai/CLIP
  !git clone https://github.com/CompVis/taming-transformers.git
  !rm -Rf clipit
  !git clone https://github.com/bharathraj-v/clipit
  !pip install ftfy regex tqdm omegaconf pytorch-lightning
  !pip install kornia==0.6.1
  !pip install imageio-ffmpeg   
  !pip install einops
  !pip install torch-optimizer
  !pip install easydict
  !pip install braceexpand
  !pip install git+https://github.com/pvigier/perlin-numpy

  !pip install svgwrite
  !pip install svgpathtools
  !pip install cssutils
  !pip install numba
  !pip install torch-tools
  !pip install visdom


  !git clone https://github.com/BachiLi/diffvg
  %cd diffvg
  !git submodule update --init --recursive
  !python setup.py install
  %cd ..
  
  !mkdir -p steps
  !mkdir -p models

output.clear()
import sys
sys.path.append("clipit")

result_msg = "setup complete"
import IPython
import os
if not os.path.isfile("first_init_complete"):
  !mkdir -p models
  os.mknod("first_init_complete")
  result_msg = "Please choose Runtime -> Restart Runtime from the menu, and then run Setup again"

js_code = f'''
document.querySelector("#output-area").appendChild(document.createTextNode("{result_msg}"));
'''
js_code += '''
for (rule of document.styleSheets[0].cssRules){
  if (rule.selectorText=='body') break
}
rule.style.fontSize = '30px'
'''
display(IPython.display.Javascript(js_code))

Dependencies for running locally:

In [ ]:
#setting up working directory
%cd bharathraj-v/
%cd Documents/
%cd Mozart/
!pwd

nvidia_output = !nvidia-smi --query-gpu=memory.total --format=noheader,nounits,csv
gpu_memory = int(nvidia_output[0])
if gpu_memory < 14000:
  warning_string = f"--> GPU check: ONLY {gpu_memory} MiB available: Please use low quality or low res <--"
  print(warning_string)
else:
  print(f"GPU check: {gpu_memory} MiB available: this should be fine")

from IPython.utils import io
with io.capture_output() as captured:
  !pip install torch==1.9.0+cu102 torchtext==0.10.0 torchvision==0.10.0+cu102 torch-optimizer==0.1.0 -f https://download.pytorch.org/whl/torch/ -f https://download.pytorch.org/whl/torchvision/
  !git clone https://github.com/openai/CLIP
  !git clone https://github.com/CompVis/taming-transformers.git
  !rm -Rf clipit
  !git clone https://github.com/bharathraj-v/clipit
  !pip install ftfy regex tqdm omegaconf pytorch-lightning
  !pip install kornia==0.6.1
  !pip install imageio-ffmpeg  
  !pip install einops
  !pip install torch-optimizer
  !pip install easydict
  !pip install braceexpand
  !pip install git+https://github.com/pvigier/perlin-numpy

  !pip install svgwrite
  !pip install svgpathtools
  !pip install cssutils
  !pip install numba
  !pip install torch-tools
  !pip install visdom


  !git clone https://github.com/BachiLi/diffvg
  %cd diffvg
  !git submodule update --init --recursive
  !python setup.py install
  %cd ..
  
  !mkdir -p steps
  !mkdir -p models

import sys
sys.path.append("clipit")

result_msg = "setup complete"
import IPython
import os
if not os.path.isfile("first_init_complete"):
  !mkdir -p models
  os.mknod("first_init_complete")
  result_msg = "Please choose Runtime -> Restart Runtime from the menu, and then run Setup again"

js_code = f'''
document.querySelector("#output-area").appendChild(document.createTextNode("{result_msg}"));
'''
js_code += '''
for (rule of document.styleSheets[0].cssRules){
  if (rule.selectorText=='body') break
}
rule.style.fontSize = '30px'
'''
display(IPython.display.Javascript(js_code))

## Importing Modules for Style Transfer

In [ ]:
import os
import tensorflow as tf
import numpy as np
import PIL.Image
import time
import functools
import tensorflow_hub as hub


##Importing Modules for Super Resolution

In [ ]:
import urllib.request
from PIL import Image
import requests
import shutil

##Importing Modules for VQGAN + Clip

In [ ]:
import torch
import clipit

##Function Definitions:

#####Style Transfer:

In [ ]:
hub_model = hub.load("https://tfhub.dev/google/magenta/arbitrary-image-stylization-v1-256/2")
def style_transfer(content_image, style_image):
    content_image = tf.convert_to_tensor(content_image, np.float32)[tf.newaxis, ...] / 255.
    style_image = tf.convert_to_tensor(style_image, np.float32)[tf.newaxis, ...] / 255.
    output = hub_model(content_image, style_image)
    stylized_image = output[0]
    return Image.fromarray(np.uint8(stylized_image[0] * 255))


#####VQGAN + Clip:

In [ ]:
def generate(prompt, generation_quality, detail):
    torch.cuda.empty_cache()
    clipit.reset_settings()
    clipit.add_settings(prompts=prompt,
                        aspect='square',
                        quality=generation_quality,
                        use_pixeldraw=False,
                        use_clipdraw=False,
                        make_video=False,
                        iterations={'draft':30, 'normal': 60, 'better': 100}[detail])
    settings = clipit.apply_settings()
    clipit.do_init(settings)
    clipit.do_run(settings)

    return 'output.png'

#####Super Resolution:

In [ ]:
def enhance(path):
  Image.fromarray(path.astype('uint8'), 'RGB').save('temp.jpg', 'jpeg')
  
  r = requests.post(
    "https://api.deepai.org/api/waifu2x",
    files={
        'image': open('temp.jpg', 'rb')
    },
    headers={'api-key': 'quickstart-QUdJIGlzIGNvbWluZy4uLi4K'}
  )
  urllib.request.urlretrieve(r.json()["output_url"],"temp.jpg")
  img = Image.open("temp.jpg")
  return img
  

#Gradio Interface:



In [ ]:
%%capture
!pip install gradio

In [ ]:
import gradio as gr

##Style Transfer:

In [ ]:
%%capture

st_description = """
<center>
<img src=https://i.imgur.com/goqyoWH.png width=700px>
</center>
"""

urllib.request.urlretrieve("https://i.imgur.com/dqpGGLm.png","stock.jpg")
urllib.request.urlretrieve("https://i.imgur.com/MHTDaZ3.png","starrynight.jpg")
starry_night = ["stock.jpg","starrynight.jpg"]
content = gr.inputs.Image(label="Content Image")
style = gr.inputs.Image(shape=(256, 256), label="Style Image")


editor = gr.Interface(style_transfer, 
                      inputs=[content, style], 
                      description = st_description,
                      examples = [starry_night],
                      article = "Style transfer is a computer vision technique"
                      +" that allows us to recompose the content of an image in the style of another",
                      outputs="image",

                      allow_flagging = "never",
                      live=False)

## VQGAN + CLIP:

In [ ]:
%%capture

ga_description = """
<center>
<img src=https://i.imgur.com/lolHhBs.png width=700px>
</center>
"""
prompt = gr.inputs.Textbox(default="Flying Cars", label="Text Prompt")
generation_quality = gr.inputs.Radio(choices=['draft', 'normal', 'better'], label="Resolution")
detail = gr.inputs.Radio(choices=['draft', 'normal', 'better'], label="Detail")


generator = gr.Interface(generate, 
                         inputs=[prompt, generation_quality, detail], 
                         outputs=['image'], 
                         description = ga_description,
                         enable_queue=True, 
                         live=False)

## Super Resolution:

In [ ]:
%%capture

sr_description = """
<center>
<img src=https://i.imgur.com/RhjYIZh.png width=700px>
</center>
"""
original = gr.Image(shape=None, image_mode="RGB", invert_colors=False, source="upload", tool="editor", type="numpy", label="content")

enhancer = gr.Interface(enhance, 
                      inputs=[original], 
                      description = sr_description,
                      article = "2x your image resolution",
                      outputs=["image"],
                      allow_flagging = "never",
                      live=False)

## Combined Interface

In [ ]:
combined = gr.TabbedInterface([generator, editor, enhancer], ["Generate Art", "Style Transfer", "Super Resolution"])
combined.launch(debug=True, share=True)

Colab notebook detected. This cell will run indefinitely so that you can see errors and logs. To turn off, set debug=False in launch().
Running on public URL: https://52131.gradio.app

This share link expires in 72 hours. For free permanent hosting, check out Spaces (https://huggingface.co/spaces)


Clean Cache:

In [ ]:
torch.cuda.empty_cache()